# 3a

In [16]:
import psycopg2
from psycopg2 import pool

class Database:

    __connection_pool = None

    @staticmethod
    def initialise(**kwargs):
        Database.__connection_pool = pool.SimpleConnectionPool(1, 10, **kwargs)

    @staticmethod
    def get_connection():
        return Database.__connection_pool.getconn()

    @staticmethod
    def return_connection(connection):
        Database.__connection_pool.putconn(connection)

    @staticmethod
    def close_all_connections():
        Database.__connection_pool.closeall()

class CursorFromConnectionPool:
    def __init__(self):
        self.conn = None
        self.cursor = None

    def __enter__(self):
        self.conn = Database.get_connection()
        self.cursor = self.conn.cursor()
        return self.cursor

    def __exit__(self, exception_type, exception_value, exception_traceback):
        if exception_value:  # This is equivalent to `if exception_value is not None`
            self.conn.rollback()
        else:
            self.cursor.close()
            self.conn.commit()
        Database.return_connection(self.conn)

    
class User:
    def __init__(self, sex, name, id=None):
        self.sex = sex
        self.name = name
        self.id = id

    def __repr__(self):
        return format(self.name)

    
    @classmethod
    def load_from_db_by_name(cls, sex):
        with CursorFromConnectionPool() as cursor:
            # Note the (email,) to make it a tuple!
            cursor.execute("SELECT name FROM pupils  WHERE sex = 'M' ", (sex,))
            user_data = cursor.fetchall()
            return user_data
    
Database.initialise(database="Learning", user="postgres", password="church75", host="localhost", port=5433)

user_from_db = User.load_from_db_by_name('sex')

print(user_from_db)




[('James',), ('John',), ('Robert',), ('Michael',), ('William',), ('David',), ('Richard',), ('Joseph',), ('Charles',), ('Thomas',), ('Christopher',), ('Weston',), ('Merle',), ('Darrel',), ('Randal',), ('Ryder',), ('Dexter',), ('Corbin',), ('Esteban',), ('Marlon',), ('Beau',), ('Blaine',), ('Stewart',), ('Branden',), ('Heath',), ('Kameron',), ('Gilberto',), ('Cayden',), ('Will',), ('Silas',), ('Aubrey',), ('Quinn',), ('Hugo',), ('Deandre',), ('Lincoln',), ('Darnell',), ('Ezekiel',), ('Amos',), ('Bert',), ('Lukas',), ('Quinton',), ('Wilfred',), ('Sawyer',), ('Kyler',), ('Jarrod',), ('Antoine',), ('Easton',), ('Percy',), ('Emil',), ('Pat',), ('Lionel',), ('Mateo',), ('Solomon',), ('Sterling',), ('Kai',), ('Ezra',), ('Timmy',), ('Moises',), ('Brenden',), ('Johnathon',), ('Dewayne',), ('Zachariah',), ('Guadalupe',), ('Jude',), ('Kaiden',), ('Mickey',), ('Tomas',), ('Bentley',), ('Reid',), ('Kendrick',), ('Stephan',), ('Cade',), ('Axel',), ('Jaiden',), ('Jerald',), ('Rocco',), ('Rocky',), ('B

# 3b

In [14]:
import psycopg2
from psycopg2 import pool

class Database:

    __connection_pool = None

    @staticmethod
    def initialise(**kwargs):
        Database.__connection_pool = pool.SimpleConnectionPool(1, 10, **kwargs)

    @staticmethod
    def get_connection():
        return Database.__connection_pool.getconn()

    @staticmethod
    def return_connection(connection):
        Database.__connection_pool.putconn(connection)

    @staticmethod
    def close_all_connections():
        Database.__connection_pool.closeall()

class CursorFromConnectionPool:
    def __init__(self):
        self.conn = None
        self.cursor = None

    def __enter__(self):
        self.conn = Database.get_connection()
        self.cursor = self.conn.cursor()
        return self.cursor

    def __exit__(self, exception_type, exception_value, exception_traceback):
        if exception_value:  # This is equivalent to `if exception_value is not None`
            self.conn.rollback()
        else:
            self.cursor.close()
            self.conn.commit()
        Database.return_connection(self.conn)

    
class User:
    def __init__(self, sex, name, id=None):
        self.sex = sex
        self.name = name
        self.id = id

    def __repr__(self):
        return format(self.name)

   
    @classmethod
    def load_from_db_by_name(cls, sex):
        with CursorFromConnectionPool() as cursor:
            # Note the (email,) to make it a tuple!
            cursor.execute("SELECT name AS female_names FROM pupils  WHERE sex = 'F' ", (sex,))
            user_data = cursor.fetchall()
            return user_data
#         return cls(sex=user_data[1], name=user_data[2], id=user_data[0])
    
Database.initialise(database="Learning", user="postgres", password="church75", host="localhost", port=5433)


user_from_db = User.load_from_db_by_name('sex')

print(user_from_db)




[('Mary',), ('Hillary',), ('Jolene',), ('Cathleen',), ('Fatima',), ('Estella',), ('Carissa',), ('Kayleigh',), ('Ora',), ('Joni',), ('Shauna',), ('Noelle',), ('Leanne',), ('Candy',), ('May',), ('Khloe',), ('Delia',), ('Brenna',), ('Karin',), ('Piper',), ('Haylee',), ('Fay',), ('Laila',), ('Lucile',), ('Tania',), ('Helena',), ('Etta',), ('Julianne',), ('Lea',), ('Sondra',), ('Macy',), ('Imogene',), ('Alyson',), ('Gwen',), ('Jayne',), ('Angelique',), ('Jodie',), ('Staci',), ('Ina',), ('Lorna',), ('Reese',), ('Polly',), ('Iva',), ('Makenna',), ('Eugenia',), ('Francesca',), ('London',), ('Cameron',), ('Lara',), ('Goldie',), ('Cleo',), ('Keira',), ('Tiana',), ('Brooklynn',), ('Daniella',), ('Jenifer',), ('Elva',), ('Luz',), ('Eliana',), ('Jewell',), ('Rosetta',), ('Susanne',), ('Kailey',), ('Eden',), ('Blanca',), ('Chasity',), ('Trudy',), ('Taryn',), ('Clare',), ('Amaya',), ('Camryn',), ('Kiana',), ('Francis',), ('Effie',), ('Aubree',), ('Tamika',), ('Kenya',), ('Maribel',), ('Tiara',), ('Li

# 4

In [7]:
import psycopg2

try:
    con = psycopg2.connect(database="Learning", user="postgres", password="church75", host="localhost", port=5433)

    with con:

        cur = con.cursor()

        cur.execute("DROP TABLE IF EXISTS Names")
        cur.execute("CREATE TABLE Names( male_name VARCHAR(255))")
    
        print("Database Table created successfully ")
except (Exception, psycopg2.Error) as error :
    print ("Unable to create databse table ", error)

Database Table created successfully 


# 5

In [8]:
import psycopg2
from psycopg2 import pool

try:
    class Database:

        __connection_pool = None

        @staticmethod
        def initialise(**kwargs):
            Database.__connection_pool = pool.SimpleConnectionPool(1, 10, **kwargs)

        @staticmethod
        def get_connection():
            return Database.__connection_pool.getconn()

        @staticmethod
        def return_connection(connection):
            Database.__connection_pool.putconn(connection)

        @staticmethod
        def close_all_connections():
            Database.__connection_pool.closeall()

    class CursorFromConnectionPool:
        def __init__(self):
            self.conn = None
            self.cursor = None

        def __enter__(self):
            self.conn = Database.get_connection()
            self.cursor = self.conn.cursor()
            return self.cursor

        def __exit__(self, exception_type, exception_value, exception_traceback):
            if exception_value:  # This is equivalent to `if exception_value is not None`
                self.conn.rollback()
            else:
                self.cursor.close()
                self.conn.commit()
            Database.return_connection(self.conn)

    
    class User:
        def __init__(self, sex, name, id=None):
            self.sex = sex
            self.name = name
            self.id = id

        def __repr__(self):
            return format(self.name)

    # inserting male list
        def save_to_db(self):
        # This is creating a new connection pool every time!
            with CursorFromConnectionPool() as cursor:
                cursor.execute("INSERT INTO Names SELECT name FROM pupils  WHERE sex = 'M' ")

    
    Database.initialise(database="Learning", user="postgres", password="church75", host="localhost", port=5433)

    user = User('id', 'sex', 'name' )

    user.save_to_db()

    # user_from_db = User.load_from_db_by_name('sex')

    print("Data inserted successfully, check your database")


except (Exception, psycopg2.Error) as error :
    print ("Unable to insert data into the table ", error)


Data inserted successfully, check your database
